In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import re
import pickle
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#loading orignal Dataset
path = "/content/drive/MyDrive/Abstractive Text Summarization/wikihowAll.csv"

wikiHowData = pd.read_csv(path)
wikiHowData.dropna(inplace=True)
wikiHowData.drop_duplicates(subset=['text'],inplace=True)  #dropping duplicates

In [ ]:
# displaying first 5 rows of Dataset

# Descriptive text => text column
#summary text => headline column

wikiHowData.head() 

,headline,title,text
0,"\nKeep related supplies in the same area.,\nMa...",How to Be an Organized Artist1,"If you're a photographer, keep all the necess..."
1,\nCreate a sketch in the NeoPopRealist manner ...,How to Create a Neopoprealist Art Work,See the image for how this drawing develops s...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",How to Be a Visual Effects Artist1,It is possible to become a VFX artist without...
3,\nStart with some experience or interest in ar...,How to Become an Art Investor,The best art investors do their research on t...
4,"\nKeep your reference materials, sketches, art...",How to Be an Organized Artist2,"As you start planning for a project or work, ..."


## Text Cleaning

In [ ]:
# dictionary that will use for expanding the contractions

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

### Cleaning Document 

In [ ]:
# Before Cleaning 
wikiHowData['text'][:10]

0     If you're a photographer, keep all the necess...
1     See the image for how this drawing develops s...
2     It is possible to become a VFX artist without...
3     The best art investors do their research on t...
4     As you start planning for a project or work, ...
5     When you finish a project, whether it sells o...
6     This should be a short video showcasing the b...
7     Networking is a great way to find new opportu...
8     Some entire movies are improvised, some plays...
9     Use your friends’ conversations to figure out...
Name: text, dtype: object

In [ ]:
import nltk
nltk.download("stopwords")

stop_words = set(stopwords.words('english')) 

def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
cleaned_text = []
for t in wikiHowData['text']:
    cleaned_text.append(text_cleaner(t))

### Cleaning Summary

In [ ]:
# Before Cleaning 
wikiHowData['headline'][:10]

0    \nKeep related supplies in the same area.,\nMa...
1    \nCreate a sketch in the NeoPopRealist manner ...
2    \nGet a bachelor’s degree.,\nEnroll in a studi...
3    \nStart with some experience or interest in ar...
4    \nKeep your reference materials, sketches, art...
5    \nKeep all of your past work organized and acc...
6    \nCreate a compelling reel or portfolio.,\nLan...
7    \nJoin a professional society.,\nEnjoy working...
8    \nMake sure you know what is expected of you.,...
9    \nMake a list of what your friends watch, read...
Name: headline, dtype: object

In [ ]:
def summary_cleaner(text):
    newString = re.sub('"','', text)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  

    return newString

In [ ]:
#Call the above function
cleaned_summary = []
for t in wikiHowData['headline']:
    cleaned_summary.append(summary_cleaner(t))
    
# updating the dataset with cleaned source document and cleaned summary
wikiHowData['cleaned_text']=cleaned_text
wikiHowData['cleaned_summary']=cleaned_summary
wikiHowData['cleaned_summary'].replace('', np.nan, inplace=True)
wikiHowData.dropna(axis=0,inplace=True)

### After Cleaning

In [ ]:
wikiHowData['cleaned_text'][:10]

0    photographer keep necessary lens cords batteri...
1    see image drawing develops step step however i...
2    possible become vfx artist without college deg...
3    best art investors research pieces art buy som...
4    start planning project work likely gathering s...
5    finish project whether sells stuff away drawer...
6    short video showcasing breadth depth skills ar...
7    networking great way find new opportunities ex...
8    entire movies improvised plays sometimes group...
9    use friends conversations figure popular pay c...
Name: cleaned_text, dtype: object

In [ ]:
wikiHowData['cleaned_summary'][:10]

0    keep related supplies in the same area make an...
1    create sketch in the neopoprealist manner of t...
2    get bachelor degree enroll in studio based pro...
3    start with some experience or interest in art ...
4    keep your reference materials sketches article...
5    keep all of your past work organized and acces...
6    create compelling reel or portfolio land an in...
7    join professional society enjoy working with t...
8    make sure you know what is expected of you sti...
9    make list of what your friends watch read and ...
Name: cleaned_summary, dtype: object

In [ ]:
# add the START and END special tokens at the beginning and end of the summary

wikiHowData['cleaned_summary'] = wikiHowData['cleaned_summary'].apply(lambda x : '<go> '+ x + ' <stop>')

In [ ]:
#Displaying 5 Source document with summary in cleaned dataset 

for i in range(5):
    print("Document:",wikiHowData['cleaned_text'][i])
    print("Summary:",wikiHowData['cleaned_summary'][i])
    print("\n")

Document: photographer keep necessary lens cords batteries quadrant home studio paints kept brushes cleaner canvas print supplies ink etc make broader groups areas supplies make finding easier limiting search much smaller area ideas include essential supplies area things use every day inspiration reference area dedicated work area infrequent secondary supplies area tucked way mean cleaning entire studio means keeping area immediately around desk easel pottery wheel etc clean night discard trash unnecessary materials wipe dirty surfaces endeavor leave workspace way sit next day start working immediately without work tidying even rest studio bit disorganized organized workspace help get business every time want make art visual people lot artist clutter comes desire keep track supplies visually instead tucked sight using jars old glasses vases cheap clear plastic drawers keep things sight without leaving strewn haphazardly ideas beyond mentioned include canvas shoe racks back door wine ra

In [ ]:
#displaying the current Dataset
wikiHowData

,headline,title,text,cleaned_text,cleaned_summary
0,"\nKeep related supplies in the same area.,\nMa...",How to Be an Organized Artist1,"If you're a photographer, keep all the necess...",photographer keep necessary lens cords batteri...,<go> keep related supplies in the same area ma...
1,\nCreate a sketch in the NeoPopRealist manner ...,How to Create a Neopoprealist Art Work,See the image for how this drawing develops s...,see image drawing develops step step however i...,<go> create sketch in the neopoprealist manner...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",How to Be a Visual Effects Artist1,It is possible to become a VFX artist without...,possible become vfx artist without college deg...,<go> get bachelor degree enroll in studio base...
3,\nStart with some experience or interest in ar...,How to Become an Art Investor,The best art investors do their research on t...,best art investors research pieces art buy som...,<go> start with some experience or interest in...
4,"\nKeep your reference materials, sketches, art...",How to Be an Organized Artist2,"As you start planning for a project or work, ...",start planning project work likely gathering s...,<go> keep your reference materials sketches ar...
...,...,...,...,...,...
215360,\nConsider changing the spelling of your name....,How to Pick a Stage Name3,"If you have a name that you like, you might f...",name like might fiddle spelling see alternate ...,<go> consider changing the spelling of your na...
215361,"\nTry out your name.,\nDon’t legally change yo...",How to Pick a Stage Name4,Your name might sound great to you when you s...,name might sound great say loud bedroom find s...,<go> try out your name don legally change your...
215362,"\nUnderstand the process of relief printing.,\...",How to Identify Prints1,Relief printing is the oldest and most tradit...,relief printing oldest traditional printing te...,<go> understand the process of relief printing...
215363,\nUnderstand the process of intaglio printing....,How to Identify Prints2,"Intaglio is Italian for ""incis­ing,"" and corr...",intaglio italian incis ing correspondingly rev...,<go> understand the process of intaglio printi...


In [ ]:
#Droping unnessary coulmns in Dataset
wikiHowData.drop(['headline', 'title', 'text'], axis=1, inplace=True) 

In [ ]:
#displaying the current Dataset
wikiHowData

,cleaned_text,cleaned_summary
0,photographer keep necessary lens cords batteri...,<go> keep related supplies in the same area ma...
1,see image drawing develops step step however i...,<go> create sketch in the neopoprealist manner...
2,possible become vfx artist without college deg...,<go> get bachelor degree enroll in studio base...
3,best art investors research pieces art buy som...,<go> start with some experience or interest in...
4,start planning project work likely gathering s...,<go> keep your reference materials sketches ar...
...,...,...
215360,name like might fiddle spelling see alternate ...,<go> consider changing the spelling of your na...
215361,name might sound great say loud bedroom find s...,<go> try out your name don legally change your...
215362,relief printing oldest traditional printing te...,<go> understand the process of relief printing...
215363,intaglio italian incis ing correspondingly rev...,<go> understand the process of intaglio printi...


In [ ]:
#saving the cleaned Dataset as a CSV file
wikiHowData.to_csv("wikiHowCleanedData.csv")